In [ ]:
import csv
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
import webdriver_manager
from webdriver_manager.firefox import GeckoDriverManager

In [ ]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

url = 'https://www.amazon.com'

driver.get(url)

In [ ]:
def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&crid=2QRDRMQSAP0KZ&sprefix=linux%2Caps%2C812&ref=nb_sb_noss_1'
    search_term = search_term.replace(' ', '+')
    return template.format(search_term)

In [ ]:
url = get_url('linux book')
print(url)

In [ ]:
driver.get(url)

Extract the Collection

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
results = soup.find_all('div', {'data-component-type': 's-search-result'})

In [ ]:
len(results)

Prototype The Record

In [ ]:
item = results[0]

In [ ]:
atag = item.h2.a

In [ ]:
description = atag.text.strip()

In [ ]:
url = 'https://www.amazon.com' + atag.get('href')

In [ ]:
price_parent = item.find('span', 'a-price')

In [ ]:
price = price_parent.find('span', 'a-offscreen').text

In [ ]:
rating = item.i.text

In [ ]:
review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'})

Generalize the Pattern

In [ ]:
def extract_record(item):
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    
    price_parent = item.find('span', 'a-price')
    price = price_parent.find('span', 'a-offscreen').text
        
    
    rating = item.i.text
    review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
    
    #rating = ''
    #review_count = ''
    
    result = (description, price, rating, review_count, url)
    
    return result

In [ ]:
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    records.append(extract_record(item))

ERROR HANDLING

In [ ]:
def extract_record(item):
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try:
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    
    result = (description, price, rating, review_count, url)
    
    return result

In [ ]:
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [ ]:
records[0]

In [ ]:
for row in records:
    print(row[1])

Getting To Next Page

In [ ]:
def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&crid=2QRDRMQSAP0KZ&sprefix=linux%2Caps%2C812&ref=nb_sb_noss_1'
    search_term = search_term.replace(' ', '+')
    
    url = template.format(search_term)
    
    url += '&page'
    
    return url

In [ ]:
import csv
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
import webdriver_manager
from webdriver_manager.firefox import GeckoDriverManager
import pandas as pd
import numpy as np

def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&crid=2QRDRMQSAP0KZ&sprefix=linux%2Caps%2C812&ref=nb_sb_noss_1'
    search_term = search_term.replace(' ', '+')
    
    url = template.format(search_term)
    
    url += '&page'
    
    return url

def extract_record(item):
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try:
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    
    result = (description, price, rating, review_count, url)
    
    return result

    
def main(search_term):
    driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    url = 'https://www.amazon.com'
    driver.get(url)

    records = []
    url = get_url(search_term)
    
    for page in range(1, 21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
                
    driver.close()
    
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'Url'])
        writer.writerows(records)
        
    #get data from results.csv
    df= pd.read_csv('./results.csv')

    #Drop unneeded column
    df.drop(['Rating','ReviewCount','Url'],axis=1, inplace=True)

    #Drop duplicate data
    df=df.drop_duplicates()

    #Convert prices from objects to floats
    df[df.columns[1:]] = df[df.columns[1:]].replace('[\$,]', '', regex=True).astype(float)

    #sort data based on the price- low to high
    df=df.sort_values('Price')

    df=df.head(50)
    
    return df

In [ ]:
main('linux book')